# AIMMS Chap.19
**2024.01.31**
- 野口翔平
- 桑田大輝

In [1]:
"""
ライブラリのインポート
 *  math     : 数値計算
 *  numpy    : 数値計算
 *  pandas   : ファイル(データフレーム)の表示
 *  gurobi   : 最適化計算  
"""
import math
import numpy as np
import pandas as pd

from gurobipy import Model, GRB, quicksum

In [2]:
"""
データの定義
 * 文章中では以下2つのファイルが定義されている
 * 各ファイルは5行のデータから構成される
 * この2つのファイルのマージを最適化によって行う
"""
IncomeData = np.array([
    [20, 10000, 3, "Commerce", 0],
    [30, 15500, 2, "Agriculture", 1000], 
    [25, 20000, 5, "Agriculture", 1000], 
    [18, 25000, 4, "Commerce", 3000],
    [32, 15000, 3, "Commerce", 500]
    ])

PopulationData = np.array([
    [25, 15000, 4, 40, 12, "M", 2],
    [30, 15000, 2, 25, 16, "M", 0],
    [18, 20000, 1, 30, 18, "F", 0],
    [27, 25000, 2, 35, 16, "F", 1],
    [25, 20000, 4, 25, 12, "M", 1],
    ])

In [3]:
# IncomeDataの表示
pd.DataFrame(IncomeData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Source of Income", "Interest Income"])

,No. of Families,Gross Family Income,No. of Familiy Members,Source of Income,Interest Income
0,20,10000,3,Commerce,0
1,30,15500,2,Agriculture,1000
2,25,20000,5,Agriculture,1000
3,18,25000,4,Commerce,3000
4,32,15000,3,Commerce,500


In [4]:
# PopulationDataの表示
pd.DataFrame(PopulationData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Head of Household(Age)", "Head of Household(Education)", "Head of Household(Sex)", "No. of Familiy Members Under 18"])

,No. of Families,Gross Family Income,No. of Familiy Members,Head of Household(Age),Head of Household(Education),Head of Household(Sex),No. of Familiy Members Under 18
0,25,15000,4,40,12,M,2
1,30,15000,2,25,16,M,0
2,18,20000,1,30,18,F,0
3,27,25000,2,35,16,F,1
4,25,20000,4,25,12,M,1


In [5]:
model = Model()

Restricted license - for non-production use only - expires 2024-10-28


In [6]:
# 添字
"""
i : supply nodes     今回の場合IncomeDataを指す
j : demand nodes     今回の場合PopurationDataを指す
"""
# パラメータ
# 2つのファイルをマージするため、共通のカラムを持つデータ(N, G, M)を抽出

## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

# 変数
"""
最適化によって計算される値
 * x_ij 5*5     : 
 * d_ij 5*5     : IncomeData[i]とPopulationData[j]の距離を格納する配列
"""
x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}


In [7]:
print(f"G_i : {G_i}")
print(f"G_j : {G_j}")
print(f"N_i : {N_i}")
print(f"N_j : {N_j}")

G_i : [10000 15500 20000 25000 15000]
G_j : [15000 15000 20000 25000 20000]
N_i : [20 30 25 18 32]
N_j : [25 30 18 27 25]


In [8]:
# 制約
"""
最適化の制約条件
 * (1) Σx_ij = N_1  : 
 * (2) Σx_ij = N_j  :
 * (3) x_ij >= 0    : x_ijが負の値を取らない。最低で0
 * (4) d            : 目的関数で用いるd_ijの定義
"""

## (1) Σx_ij = N_i
model.addConstrs((quicksum(x_ij[i, j] for j in range(5)) - N_i[i] == 0 for i in range(5)))

## (2) Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in range(5)) - N_j[j] == 0 for j in range(5)))

## (3) x_ij >= 0
model.addConstrs((x_ij[i, j] >= 0 for j in range(5) for i in range(5)))

## (4) d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in range(5) for j in range(5))

# 最適化の実行
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i in range(5) for j in range(5)),GRB.MINIMIZE); #目的関数の追加
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 50 columns and 100 nonzeros
Model fingerprint: 0xf6de555c
Model has 25 quadratic objective terms
Variable types: 25 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+01]
Found heuristic solution: objective 252.9675974
Presolve removed 50 rows and 25 columns
Presolve time: 0.00s
Presolved: 10 rows, 25 columns, 50 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)

Root relaxation: objective 1.442615e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0    

In [9]:
# 最適値の出力
print("Opt.value=",model.objval)

Opt.value= 144.26148655218032


In [10]:
# 最適解の出力 (x_ij)
for i in range(5): 
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i,j].X)

x_ij[ 0 ][ 0 ]= 20.0
x_ij[ 0 ][ 1 ]= -0.0
x_ij[ 0 ][ 2 ]= -0.0
x_ij[ 0 ][ 3 ]= -0.0
x_ij[ 0 ][ 4 ]= -0.0
x_ij[ 1 ][ 0 ]= -0.0
x_ij[ 1 ][ 1 ]= 30.0
x_ij[ 1 ][ 2 ]= -0.0
x_ij[ 1 ][ 3 ]= -0.0
x_ij[ 1 ][ 4 ]= -0.0
x_ij[ 2 ][ 0 ]= -0.0
x_ij[ 2 ][ 1 ]= -0.0
x_ij[ 2 ][ 2 ]= -0.0
x_ij[ 2 ][ 3 ]= -0.0
x_ij[ 2 ][ 4 ]= 25.0
x_ij[ 3 ][ 0 ]= -0.0
x_ij[ 3 ][ 1 ]= -0.0
x_ij[ 3 ][ 2 ]= -0.0
x_ij[ 3 ][ 3 ]= 18.0
x_ij[ 3 ][ 4 ]= 0.0
x_ij[ 4 ][ 0 ]= 5.0
x_ij[ 4 ][ 1 ]= -0.0
x_ij[ 4 ][ 2 ]= 18.0
x_ij[ 4 ][ 3 ]= 9.0
x_ij[ 4 ][ 4 ]= -0.0


In [11]:
# 最適解の出力 (d_ij)
for i in range(5): 
    for j in range(5):
        print("d_ij[",i,"][",j,"]=",d_ij[i,j].X)

d_ij[ 0 ][ 0 ]= 1.3858946059547992
d_ij[ 0 ][ 1 ]= 1.3858946059547992
d_ij[ 0 ][ 2 ]= 2.7717892119095984
d_ij[ 0 ][ 3 ]= 3.401771452500264
d_ij[ 0 ][ 4 ]= 2.3537115992409285
d_ij[ 1 ][ 0 ]= 1.693873383281096
d_ij[ 1 ][ 1 ]= 0.1098370677198228
d_ij[ 1 ][ 2 ]= 1.3005708021304023
d_ij[ 1 ][ 3 ]= 2.0869042866766327
d_ij[ 1 ][ 4 ]= 1.9581474801994003
d_ij[ 2 ][ 0 ]= 1.3858946059547992
d_ij[ 2 ][ 1 ]= 2.763148489151519
d_ij[ 2 ][ 2 ]= 3.3806170189140663
d_ij[ 2 ][ 3 ]= 2.763148489151519
d_ij[ 2 ][ 4 ]= 0.8451542547285166
d_ij[ 3 ][ 0 ]= 2.1967413543964556
d_ij[ 3 ][ 1 ]= 2.7717892119095984
d_ij[ 3 ][ 2 ]= 2.763148489151519
d_ij[ 3 ][ 3 ]= 1.6903085094570331
d_ij[ 3 ][ 4 ]= 1.0983706771982278
d_ij[ 4 ][ 0 ]= 0.8451542547285166
d_ij[ 4 ][ 1 ]= 0.8451542547285166
d_ij[ 4 ][ 2 ]= 2.0158276220132887
d_ij[ 4 ][ 3 ]= 2.3537115992409285
d_ij[ 4 ][ 4 ]= 1.3858946059547992


In [12]:
# 最適化した結果を元に、マージ後のデータを表示
MergeData = [] # 初期化
for i in range(5):
    for j in range(5):
        if (x_ij[i, j].X > 0):
            MergeData.append(
                np.concatenate([
                    np.array([x_ij[i, j].X, (G_i[i] + G_j[j]) / 2, max(M_i[i], M_j[j])]).astype(str),
                    IncomeData[i][3:].astype(str),
                    PopulationData[j][3:].astype(str)
                ])
            )
pd.DataFrame(MergeData, columns=["No. of Families", "Gross Family Income", "No. of Familiy Members", "Source of Income", "Interest Income", "Head of Household(Age)", "Head of Household(Education)", "Head of Household(Sex)", "No. of Familiy Members Under 18"])

,No. of Families,Gross Family Income,No. of Familiy Members,Source of Income,Interest Income,Head of Household(Age),Head of Household(Education),Head of Household(Sex),No. of Familiy Members Under 18
0,20.0,12500.0,4.0,Commerce,0,40,12,M,2
1,30.0,15250.0,2.0,Agriculture,1000,25,16,M,0
2,25.0,20000.0,5.0,Agriculture,1000,25,12,M,1
3,18.0,25000.0,4.0,Commerce,3000,35,16,F,1
4,5.0,15000.0,4.0,Commerce,500,40,12,M,2
5,18.0,17500.0,3.0,Commerce,500,30,18,F,0
6,9.0,20000.0,3.0,Commerce,500,35,16,F,1


## Exercize3

**上記のEx1の方法だと計算量が多い。調べる計算量を制限し、行数が増えた場合でも計算できるようにしたい。**
**アルゴリズムはPDF内のフローチャートを参照**

In [13]:
# 添字
"""
i : supply nodes 
j : demand nodes 
"""
# パラメータ
## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

# 変数

x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}

In [14]:
# Initialization (1) N_i, N_jの全ての要素からNを引く

sortedRankI = [0, 4, 1, 2, 3] # IncomeDataの行の並び替え (昇順)
sortedRankP = [0, 1, 2, 4, 3] # PopulationDataの行の並び替え (昇順)

N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

I = iter(sortedRankI) 
J = iter(sortedRankP)
i_star = next(I)
j_star = next(J) 
S = []
x_ij = [[0 for _ in range(len(sortedRankI))] for _ in range(len(sortedRankP))]

try:
    while True:
        while N_i[i_star]<=0:
            i_star = next(I)
        while N_j[j_star]<=0:
            j_star = next(J)
        N_star = min(N_i[i_star], N_j[j_star])
        x_ij[i_star][j_star] = N_star
        N_i -= N_star
        N_j -= N_star
        S.append((i_star, j_star))
except StopIteration: 
    print("Finished.")

print(" ============= 表示 ============= ")
print(S)
for i in range(len(sortedRankI)): 
    for j in range(len(sortedRankP)):
        print("x_ij[",i,"][",j,"]=",x_ij[i][j])

Finished.
 ============= 表示 ============= 
[(0, 0), (4, 0), (4, 1)]
x_ij[ 0 ][ 0 ]= 20
x_ij[ 0 ][ 1 ]= 0
x_ij[ 0 ][ 2 ]= 0
x_ij[ 0 ][ 3 ]= 0
x_ij[ 0 ][ 4 ]= 0
x_ij[ 1 ][ 0 ]= 0
x_ij[ 1 ][ 1 ]= 0
x_ij[ 1 ][ 2 ]= 0
x_ij[ 1 ][ 3 ]= 0
x_ij[ 1 ][ 4 ]= 0
x_ij[ 2 ][ 0 ]= 0
x_ij[ 2 ][ 1 ]= 0
x_ij[ 2 ][ 2 ]= 0
x_ij[ 2 ][ 3 ]= 0
x_ij[ 2 ][ 4 ]= 0
x_ij[ 3 ][ 0 ]= 0
x_ij[ 3 ][ 1 ]= 0
x_ij[ 3 ][ 2 ]= 0
x_ij[ 3 ][ 3 ]= 0
x_ij[ 3 ][ 4 ]= 0
x_ij[ 4 ][ 0 ]= 5
x_ij[ 4 ][ 1 ]= 5
x_ij[ 4 ][ 2 ]= 0
x_ij[ 4 ][ 3 ]= 0
x_ij[ 4 ][ 4 ]= 0


In [15]:
# Initialization (2) N_i[i_star], N_j[j_star]からNを引く

sortedRankI = [0, 4, 1, 2, 3] # IncomeDataの行の並び替え (昇順)
sortedRankP = [0, 1, 2, 4, 3] # PopulationDataの行の並び替え (昇順)


N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

I = iter(sortedRankI) 
J = iter(sortedRankP)
i_star = next(I)
j_star = next(J) 
S = []
x_ij = [[0 for _ in range(len(sortedRankI))] for _ in range(len(sortedRankP))]

try:
    while True:
        while N_i[i_star]<=0:
            i_star = next(I)
        while N_j[j_star]<=0:
            j_star = next(J)
        N_star = min(N_i[i_star], N_j[j_star])
        x_ij[i_star][j_star] = N_star
        N_i[i_star] -= N_star
        N_j[j_star] -= N_star
        S.append((i_star, j_star))
except StopIteration: 
    print("Finished.")
print(" ============= 表示 ============= ")
print(S)
for i in range(len(sortedRankI)): 
    for j in range(len(sortedRankP)):
        print("x_ij[",i,"][",j,"]=",x_ij[i][j])

Finished.
 ============= 表示 ============= 
[(0, 0), (4, 0), (4, 1), (1, 1), (1, 2), (1, 4), (2, 4), (2, 3), (3, 3)]
x_ij[ 0 ][ 0 ]= 20
x_ij[ 0 ][ 1 ]= 0
x_ij[ 0 ][ 2 ]= 0
x_ij[ 0 ][ 3 ]= 0
x_ij[ 0 ][ 4 ]= 0
x_ij[ 1 ][ 0 ]= 0
x_ij[ 1 ][ 1 ]= 3
x_ij[ 1 ][ 2 ]= 18
x_ij[ 1 ][ 3 ]= 0
x_ij[ 1 ][ 4 ]= 9
x_ij[ 2 ][ 0 ]= 0
x_ij[ 2 ][ 1 ]= 0
x_ij[ 2 ][ 2 ]= 0
x_ij[ 2 ][ 3 ]= 9
x_ij[ 2 ][ 4 ]= 16
x_ij[ 3 ][ 0 ]= 0
x_ij[ 3 ][ 1 ]= 0
x_ij[ 3 ][ 2 ]= 0
x_ij[ 3 ][ 3 ]= 18
x_ij[ 3 ][ 4 ]= 0
x_ij[ 4 ][ 0 ]= 5
x_ij[ 4 ][ 1 ]= 27
x_ij[ 4 ][ 2 ]= 0
x_ij[ 4 ][ 3 ]= 0
x_ij[ 4 ][ 4 ]= 0


In [16]:
sortedRankI = [3, 2, 1, 4, 0] # IncomeDataの行の並び替え (降順)
sortedRankP = [3, 4, 2, 1, 0] # PopulationDataの行の並び替え (降順)

## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member

def SubModel(S , index="None"):
    model = Model(f"SubModel: ({index})")

    x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
    d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}

    ## No. of Families
    N_i = IncomeData[:, 0].astype(int)
    N_j = PopulationData[:, 0].astype(int)

    ## Gross Family Income
    G_i = IncomeData[:, 1].astype(int)
    G_j = PopulationData[:, 1].astype(int)

    ## No. of Family Members
    M_i = IncomeData[:, 2].astype(int)
    M_j = PopulationData[:, 2].astype(int)

    ## 分散
    s_G = np.var(np.concatenate([G_i, G_j])) # Gross
    s_M = np.var(np.concatenate([M_i, M_j])) # Member

    # 制約
    # ## Σx_ij = N_i
    for k in range(5):
        model.addConstr(quicksum(x_ij[i, j] for i, j in S if k == i) - N_i[k] == 0)

    ## Σx_ij = N_j
    for k in range(5):
        model.addConstr(quicksum(x_ij[i, j] for i, j in S if k == j) - N_j[k] == 0)
        
    # x_ij >= 0
    model.addConstrs(x_ij[i, j] >= 0 for i, j in S)

    # d
    model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i, j in S)

    model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i, j in S),GRB.MINIMIZE); #目的関数の追加

    model.optimize()

    # 最適値を取得して返す
    if model.status == GRB.OPTIMAL:
        # x_ijの最適値を取得
        optimal_x_ij = {(i, j): x_ij[i, j].X for i in range(5) for j in range(5)}
        
        # d_ijの最適値を取得
        optimal_d_ij = {(i, j): d_ij[i, j].X for i in range(5) for j in range(5)}

        return optimal_x_ij, optimal_d_ij

In [17]:
"""
Ex.3 フローチャート内のアルゴリズム
 * 初期化をし暫定のSを取得し、暫定なSで最適化を行い、徐々にSを増やしていく手法。
 * ループの回数は多くなるが、長いSで回すより計算オーダーが少なくなる??
 (1)
 (2)
 (3)
"""

S = [(3, 3), (2, 3), (2, 4), (1, 4), (1, 2), (1, 1), (4, 1), (4, 0), (0, 0)] # initialize(2)より
x_ij, d_ij = SubModel(S, index=0)
print(x_ij)
print(d_ij)
CandidateCount = None
index = 0
while True:
    index += 1
    SearchCount = 0
    CandidateCount=0
    for i in sortedRankI: # [3, 2, 1, 4, 0]
        for j in sortedRankP: # [3, 4, 2, 1, 0]
            if ((d_ij[i, j]
                - sum(d_ij[i_star, j] for i_star in range(5) if i_star != i) 
                - sum(d_ij[i, j_star] for j_star in range(5) if j_star != j)) < 0 ):
                if (i, j) not in S:
                    S.append((i, j))
                CandidateCount += 1
            SearchCount += 1
    if CandidateCount == 25: break
    if index > 10: break # 無限ループにならないように
    print(f" =========== index: {index} =========== ")
    print(CandidateCount)
    print(SearchCount)
    print(S)
    x_ij, d_ij  = SubModel(S, index=index)
    print(x_ij)
    print(d_ij)
    print(f"=========== finish: {index} =========== ")
print(CandidateCount)
print(SearchCount)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 50 columns and 36 nonzeros
Model fingerprint: 0x799b72c4
Model has 9 quadratic objective terms
Variable types: 25 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+01]
Found heuristic solution: objective 164.9422989
Presolve removed 28 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 164.942 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.649422988820e+02, best bound 1.649422988820e+02, gap 0.0000%
{(0, 0): 20.0, (0, 1): -0.0, (0, 2): -0.0

In [18]:
for i in range(5): 
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i, j])

x_ij[ 0 ][ 0 ]= 20.0
x_ij[ 0 ][ 1 ]= -0.0
x_ij[ 0 ][ 2 ]= -0.0
x_ij[ 0 ][ 3 ]= -0.0
x_ij[ 0 ][ 4 ]= -0.0
x_ij[ 1 ][ 0 ]= -0.0
x_ij[ 1 ][ 1 ]= 30.0
x_ij[ 1 ][ 2 ]= -0.0
x_ij[ 1 ][ 3 ]= -0.0
x_ij[ 1 ][ 4 ]= -0.0
x_ij[ 2 ][ 0 ]= -0.0
x_ij[ 2 ][ 1 ]= -0.0
x_ij[ 2 ][ 2 ]= -0.0
x_ij[ 2 ][ 3 ]= -0.0
x_ij[ 2 ][ 4 ]= 25.0
x_ij[ 3 ][ 0 ]= -0.0
x_ij[ 3 ][ 1 ]= -0.0
x_ij[ 3 ][ 2 ]= -0.0
x_ij[ 3 ][ 3 ]= 18.0
x_ij[ 3 ][ 4 ]= 0.0
x_ij[ 4 ][ 0 ]= 5.0
x_ij[ 4 ][ 1 ]= -0.0
x_ij[ 4 ][ 2 ]= 18.0
x_ij[ 4 ][ 3 ]= 9.0
x_ij[ 4 ][ 4 ]= -0.0
